In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import geopandas as gpd


In [ ]:
bf = pd.read_csv('comb_myfunda[1-1800].csv')
org_len = bf.__len__()
print(org_len)
bf.head(3)

In [ ]:
cities_df = pd.read_csv('cities.csv')

cities_df.info()

In [ ]:
bf.info()

In [ ]:
df = bf.drop(bf[bf['Price'].isna() | bf['Energylabel'].isna() | bf['Build Year'].isin(['Voor 1906','Na 2020'])].index).reset_index(drop=True)

dropped = bf[bf['Price'].isna() | bf['Energylabel'].isna()  | bf['Build Year'].isin(['Voor 1906','Na 2020'])].__len__()
print(f'Percent dropped: {dropped/len(bf)*100:.2f}%')

In [ ]:
df.info()

In [ ]:
# Processing

## Province - Nothing

## City
df['City'] = df['City'].str.split(' \(', expand=True)[0]
df['City'] = df['City'].str.replace('&#39;', "'").str.replace('&#226;', 'â').str.replace('&#235;', 'ë').str.replace('&#251;','û').str.replace('&#233;','é')

## Street - Nothing

## Price
df['Price'] = df['Price'].astype(np.uint32);

## Residential house type
apartment_tags = df['Residential house type'][df['Residential house type'].str.contains('flat|portiek|appartement|Portiek')].unique()

                                                                                                            #? Worse way:
df['Apartment'] = df['Residential house type'].isin(apartment_tags).astype(int)
df['Detached'] = 1 - df['Apartment']                                                                       # def check_type(row):
                                                                                                            #     global appartment_tags
                                                                                                            #     if row['Residential house type'] not in appartment_tags:
                                                                                                            #         row['Detached'] = 1
                                                                                                            #         row['Appartment'] = 0
                                                                                                            #     else:
                                                                                                            #         row['Detached'] = 0
                                                                                                            #         row['Appartment'] = 1
                                                                                                            #     return row
                                                                                                            # df.apply(check_type, axis=1)

##  Age
build_type = {name: val for val, name in enumerate(df['Age'].unique())}
build_type_reverse = {item: key for key, item in build_type.items()}

df['Age'] = df['Age'].apply(lambda x:build_type[x]).astype(np.uint8)
df.rename(columns={'Age': 'New'}, inplace=True)

## Build Year
df['Build Year'][df['Build Year'].str.len() > 4] = df['Build Year'][df['Build Year'].str.len() > 4].str.split('-', expand=True).astype(int).mean(axis=1).astype(int);
df['Build Year'] = df['Build Year'].astype(np.uint16);

## Number of Rooms
def get_room_counts(x):
    matches = re.findall('(\d*)\s', x)
    if len(matches) < 2:
        return pd.Series([matches[0], 0])
    else:
        return pd.Series([matches[0], matches[2]])

df[['Total Rooms', 'Bedrooms Count']] = df['Number of Rooms'].apply(get_room_counts)
df['Total Rooms'] = df['Total Rooms'].astype(int)
df['Bedrooms Count'] = df['Bedrooms Count'].astype(int)

### Floors
df['Number of Floors'] = df['Number of Floors'].replace(dict.fromkeys(['een','en',',',' '], ''), regex=True)
df[['Floors', 'Extra Floors']] = df['Number of Floors'].str.split('woonlag|woonlaag', expand=True)
df = df.join(pd.get_dummies(df['Extra Floors'][df['Extra Floors'] != '']).set_axis(['kelder','vliering','vliering en kelder','zolder','zolder en kelder','zolder met vliering','zolder met vliering en kelder'], axis=1))
df[['kelder','vliering','vliering en kelder','zolder','zolder en kelder','zolder met vliering','zolder met vliering en kelder']] = df[['kelder','vliering','vliering en kelder','zolder','zolder en kelder','zolder met vliering','zolder met vliering en kelder']].replace(np.nan, 0)

df['kelder'] = df['kelder'] + df['zolder en kelder'] + df['vliering en kelder'] + df['zolder met vliering en kelder']
df['vliering'] = df['vliering'] + df['vliering en kelder'] + df['zolder met vliering'] + df['zolder met vliering en kelder']
df['zolder'] = df['zolder'] + df['zolder en kelder'] + df['zolder met vliering'] + df['zolder met vliering en kelder']

df['Floors'] = df['Floors'].astype(np.uint8)

## Number of Toilets
df[['Bathrooms', 'Separate Toilets']] = df['Number of Toilets'].str.replace(' aparte toiletten| apart toilet', '').str.split(' badkamer en | badkamers en | badkamer| badkamers',expand=True)
df['Bathrooms'][df['Bathrooms'].isna()] = 0
df['Bathrooms'] = df['Bathrooms'].astype(int)
df['Separate Toilets'][(df['Separate Toilets'] == '') | (df['Separate Toilets'] == 's')|(df['Separate Toilets'].isna())] = 0
df['Separate Toilets'] = df['Separate Toilets'].astype(int)
df['Number of Toilets'] = df['Bathrooms'] + df['Separate Toilets']



## Energy label
energy_label = {'Niet verplicht': 0, 'G': 1, 'F': 2, 'E': 3, 'D': 4, 'C': 5, 'B': 6, 'A++++': 11, 'A+++': 10, 'A++': 9, 'A+': 8, 'A': 7, 'A+++++': 12}
energy_label_reverse = {item: key for key, item in energy_label.items()}

df['Energylabel'] = df['Energylabel'].apply(lambda x: energy_label[x]).astype(np.uint8)

## Garage
df['Garage'][df['Garage'].isna()] = 0
df['Garage'][~df['Garage'].isin([0])] = 1
df['Garage'] = df['Garage'].astype(np.int8)

## Backyardsize
from math import prod

df['Backyard size'][df['Backyard size'].isna()] = '0'
df['Backyard size'] = df['Backyard size'].str.split(' m² \(', expand=True)[0].str.replace('.', '').str.replace(',', '.')
df['Backyard size'][df['Backyard size'].str.contains('meter')] = df['Backyard size'][df['Backyard size'].str.contains('meter')].str.findall('[\d\.]+').apply(lambda x: prod([float(j) for j in x]))
df['Backyard size'] = df['Backyard size'].astype(int)

## LAt Lon
df['Lat'] = df['City'].map(cities_df.set_index('City')['Lat'])
df['Lon'] = df['City'].map(cities_df.set_index('City')['Lon'])


del df['Number of Floors']
del df['Residential house type']


df.info()

In [ ]:
provinces = gpd.read_file('NLD_adm1.shp')
provinces = provinces.iloc[[0,1,2,3,4,6,7,8,9,10,11,13], :].reset_index(drop=True)
merged = provinces.join(pd.DataFrame(df.groupby('Province')['Price'].mean()).reset_index(drop=True))

In [ ]:
fig, ax = plt.subplots(figsize=(9, 10))

cutoff = df['Price'].mean()


sns.scatterplot(x=df['Lon'], y=df['Lat'], data=df[df['Price'] <= cutoff], hue=df[df['Price'] <=cutoff]['Price'], palette='winter', alpha=0.7, edgecolor='none', ax=ax)
sns.scatterplot(x=df['Lon'], y=df['Lat'], data=df[df['Price'] >= cutoff], hue=df[df['Price'] >=cutoff]['Price'], palette='Wistia', alpha=0.35, edgecolor='none', ax=ax)

# city_data = df.groupby('City').agg({'Lon': 'mean', 'Lat': 'mean', 'Price': 'mean'}).reset_index(drop=True)

# plt.figure(figsize=(10,8))
# sns.scatterplot(x=city_data['Lon'], y=city_data['Lat'], data=city_data[city_data['Price'] <= cutoff], hue=city_data['Price'].apply(lambda x: x <= cutoff)
# , palette='winter', edgecolor='none', ax =ax)



In [ ]:
fig, ax = plt.subplots(figsize=(9, 10))
merged.plot(column="Price", cmap="OrRd", linewidth=0.7, ax=ax, edgecolor="white", legend=True)
ax.set_title("Dutch Average House Prices by Province", fontsize=16);
ax.set_axis_off()
plt.show()

df.groupby('Province')['Price'].count().plot(kind='bar', title='Number of houses for sale per province')


In [ ]:
plt.figure(figsize=(15,10))
corr_data = df.select_dtypes(exclude=['object']).corr().round(2)
mask = np.zeros_like(corr_data,dtype=np.bool_)
mask[np.triu_indices_from(mask)] = True


sns.heatmap(corr_data,mask=mask, annot=True)

In [ ]:
sx = sns.displot(df['Price'])
plt.axvline(x=df['Price'].mean(),color='red', linestyle='--', linewidth=2)

sx.figure.set_figwidth(25)

In [ ]:
diff = df.groupby('Apartment')['Price'].mean()
if diff[0] > diff[1]:
    print(f'The average price of an apartment is {(diff[0] - diff[1])/max(diff[0], diff[1])*100:.2f}% cheaper than the average price of a detached house')
else:
    print(f'The average price of a detached house is {(diff[0] - diff[1])/max(diff[0], diff[1])*100:.2f}% cheaper than the average price of an apartment')    


In [ ]:
af = pd.DataFrame((df.groupby(['City']).filter(lambda x: set(x['Apartment']) == {0, 1})).groupby(['City', 'Apartment'])[['Price']].mean())
af1 = pd.pivot_table(af, index=['City'], columns=['Apartment'])

pop_by_province = ['Emmen', 'Almere', 'Leeuwarden', 'Nijmegen', 'Groningen', 'Maastricht', 'Eindhoven', 'Amsterdam', 'Enschede', 'Rotterdam', 'Utrecht', 'Terneuzen']

af1.loc[pop_by_province].plot(kind='bar', figsize=(10,7), title='House Price Detached vs Apartment in most populous city per province');

In [ ]:
byc = df.groupby('Build Year')['Energylabel'].count()[df.groupby('Build Year')['Energylabel'].count() > 10]
byc.plot(figsize=(15,7), xticks=byc.index[::10], title='Number of listings of buildings by year')
plt.axvline(x=1939,color='red', linestyle='--', linewidth=1)
plt.text(1936, 350, 'WW2', rotation=90, color='red');
plt.axvline(x=1980,color='red', linestyle='--', linewidth=1);
plt.text(1977, 10, 'Dutch Housing Crisis', rotation=90, color='red');
plt.axvline(x=2008,color='red', linestyle='--', linewidth=1)
plt.text(2005, 50, '2008 Financial Crisis', rotation=90, color='red');

In [ ]:
bye = df.groupby('Build Year')['Energylabel'].mean()[df.groupby('Build Year')['Energylabel'].count() > 10]
byex = bye.plot(figsize=(15,7), xticks=bye.index[::10], title='Average energy label by year')
byex.set_yticklabels(pd.Series(byex.get_yticks()).map(lambda x: energy_label_reverse[x]));

In [ ]:
temp = df[df['Detached'] == 1]['Backyard size'].mean().round(2)

print(f'Average backyard size for a Detached house is {temp} sqaure meters')